# Библиотека requests
 Добавить страницу в мои закладки
✍ В стандартной библиотеке Python для отправки веб-запросов существует функция urllib2, но большинство разработчиков используют стороннюю библиотеку requests (c англ. запросы), потому что её работа более стабильна, а созданный с её помощью код получается проще. Поэтому мы будем работать с библиотекой requests, а urllib2 рассматривать не будем.

Познакомимся с библиотекой requests, решив простую задачу — получить значения курсов валют. Курс валют — полезная и регулярно обновляемая информация, но каждый раз в ручном режиме получать информацию о курсе интересующей валюты трудоёмко.

Разработаем код, так называемый скрипт (англ. script, рус. сценарий), — небольшую программу, которая содержит последовательность действий для автоматического выполнения задачи.

С помощью скрипта мы будем в удобном виде выгружать информацию по курсам валют с заранее выбранного сайта.

Один из сайтов в интернете, на котором информация о курсах валют дублирует информацию с сайта Центрального Банка России, — ресурс Курсы валют ЦБ РФ в XML и JSON. На данном ресурсе информация о курсах валют представлена в разных форматах, в том числе и в структурированном JSON-формате, методы работы с которым мы изучили в одном из предыдущих модулей.

Перед началом работы библиотеку requests потребуется установить. Например, в Jupyter Notebook это делается с помощью такой команды:

In [25]:
!pip install requests 



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Как только библиотека установлена, импортируем её и отправим наш первый запрос к ресурсу Курсы валют ЦБ РФ в XML и JSON. Используем метод get() из библиотеки requests, передав ему соответствующий URL —  https://www.cbr-xml-daily.ru/daily_json.js:

In [26]:
import requests # Импортируем библиотеку requests
url = 'https://www.cbr-xml-daily.ru/daily_json.js' # Определяем значение URL страницы для запроса
response = requests.get(url) # Делаем GET-запрос к ресурсу и результат ответа сохраняем в переменной response
print(response)
#Код ответа в виде числовой переменной можно получить с помощью метода status_code:
print(response.status_code)

<Response [200]>
200


## РАБОТАЕМ С ОТВЕТОМ

Мы сделали запрос и получили корректный ответ (код статуса — 200). Дальнейшую работу производим с результатом запроса к ресурсу Курсы валют ЦБ РФ в XML и JSON.

?
Как получить доступ ко всей информации, которую содержит ответ?

Текст ответа хранится в атрибуте text. Выведем значение атрибута на экран и посмотрим на его содержимое:

In [28]:
 # Выводим содержимое атрибута text переменной response на экран
print(type(response))

#Как правило, при работе над реальным проектом на этапе получения данных мы уже понимаем, с какими форматами
#  данных нам придётся работать. На предлагаемом для работы ресурсе информация есть как в JSON-формате, так и в XML.
#  По нашему запросу ресурс возвращает информацию в JSON-формате, однако в настоящий момент результат хранится как 
# единая строка. Проверить тип данных полученного ответа можно, воспользовавшись функцией type().
#Для того чтобы удобно было работать с полученной информацией, нам необходимо преобразовать строку в словарь. 
# В объект ответа Response  из библиотеки requests уже встроен метод json() .
#Импортируем функцию pprint(), применим к полученному ответу метод json() и выведем полученный результат на экран:
from pprint import pprint # Импортируем функцию pprint()
currencies = response.json() # Применяем метод json()
 # Выводим результат на экран)

#Теперь данные находятся в словаре и можно легко получать необходимые значения.

#Например, по ключу Valute мы можем обратиться к вложенному словарю, который 
#содержит информацию о мировых валютах. Выведем на экран, например, информацию о евро (EUR):
pprint(currencies['Valute']['EUR'])
print(currencies['Valute']['CZK']['Name'])

<class 'requests.models.Response'>
{'CharCode': 'EUR',
 'ID': 'R01239',
 'Name': 'Евро',
 'Nominal': 1,
 'NumCode': '978',
 'Previous': 63.3008,
 'Value': 63.3882}
Чешских крон


# Парсинг сайтов
 Добавить страницу в мои закладки
✍ Ресурс, с которым мы работали в предыдущем юните, возвращал ответ о текущем курсе валют в удобном, структурированном формате, из которого было легко извлечь необходимую информацию. Довольно часто для получения информации приходится обращаться напрямую к HTML-страницам.

Для примера рассмотрим страницу, содержащую статью с информацией о присуждении Нобелевской премии по экономике в 2021 году, и попробуем извлечь из неё заголовок статьи, опубликованной на странице, дату публикации, а также текст статьи.

Получить содержимое страницы в большинстве случаев несложно, гораздо труднее извлечь из HTML-кода нужную информацию.

?
Что собой представляет HTML?

## ОСНОВЫ HTML

HTML (англ. HyperText Markup Language, рус. язык гипертекстовой разметки) — стандартизированный язык разметки документов в интернете. Большинство веб-страниц содержат описание разметки на языке HTML. Язык HTML интерпретируется браузерами. Полученный в результате интерпретации текст отображается на экране монитора компьютера или мобильного устройства.

HTML позволяет создавать макет страницы, разбивая её на блоки: мы можем поместить содержимое посередине страницы, сбоку и т. п.

Кроме того, HTML используется для описания форматирования. Например, с его помощью мы можем указать, какая часть текста должна отображаться крупным шрифтом как заголовок, какая — курсивом, а какая — как обычный текст.

HTML является близким родственником уже знакомого вам формата XML. Разметка на языке HTML делается с помощью так называемых тегов, которые помещаются в угловые скобки, и применяется к элементам, заключённым внутри них. Посмотрите на примеры:

h2 Это заголовок второго уровня /h2
div А это обычный текст /div
У корректной HTML-страницы есть заголовок и тело страницы. В заголовке (в тегах head … /head)  размещается техническая информация, подключаются скрипты и стили. В теле body … /body находятся текст и данные, которые непосредственно отображаются на странице в браузере.

Разметка небольшой страницы выглядит примерно так:


Обратите внимание, что теги образуют иерархическую структуру, то есть одни теги расположены внутри других. В примере выше тег p … /p находится внутри тега body … /body.

Кроме того, у тегов могут быть атрибуты, которые пишутся внутри открывающегося тега. Самые популярные атрибуты — это class и id:

h1 id="big-title"> Заголовок страницы /h1
p class="red-back"> Какой-то текст /p
Изучение языка HTML находится вне рамок этого курса, но для того, чтобы собирать информацию с веб-страниц, нет необходимости хорошо знать HTML. Достаточно понимать, что:

существуют теги с разными именами;
у тегов бывают атрибуты, такие как class и id;
теги образуют иерархическую структуру, то есть одни теги вложены в другие.
ДОПОЛНИТЕЛЬНО

Вы можете ознакомиться с информацией о HTML в справочнике, перейдя по ссылке.

ПОЛУЧАЕМ СОДЕРЖИМОЕ ВЕБ-СТРАНИЦЫ

Получим HTML-код интересующей нас страницы.

Для этого отправим GET-запрос с помощью библиотеки requests и метода get() и посмотрим на текст ответа на наш запрос (как мы помним, он содержится в атрибуте text):

In [ ]:
import requests # Импортируем библиотеку requests
url = 'https://nplus1.ru/news/2021/10/11/econobel2021' # Определяем адрес страницы
response = requests.get(url)  # Выполняем GET-запрос
print(response.text)  # Выводим содержимое атрибута text
#Ответ содержит HTML-код страницы, к которой мы обратились.
#В отличие от предыдущего примера, где ответ возвращался в JSON-формате, мы не можем так просто
#  преобразовать HTML-код в словарь и извлечь необходимую нам информацию.
#Для решения таких задач в Python существует специальная библиотека BeautifulSoup, о работе с которой
#  мы поговорим в следующем юните.

# Библиотека BeautifulSoup
 Добавить страницу в мои закладки
✍ Для поиска необходимых нам данных мы будем использовать библиотеку BeautifulSoup, которая позволяет по названию тегов и их атрибутов получать содержащийся в них текст.

BeautifulSoup не является частью стандартной библиотеки, поэтому для начала её нужно установить. Например, в Jupyter Notebook это делается с помощью такой команды:

In [32]:
# Устанавливаем библиотеку BeautifulSoup
!pip install beautifulsoup4 
#После установки импортируем библиотеку в наш код:

from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
#Теперь мы можем извлекать данные из любой веб-страницы.

#Ранее мы уже получили содержимое страницы с помощью GET-запроса и сохранили информацию в переменной response , 
# теперь создадим объект BeautifulSoup с именем page, указывая в качестве параметра html.parser.

#Для примера получим информацию o title (с англ. заголовок) — это строка, которая отображается на вкладке браузера:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
url = 'https://nplus1.ru/news/2021/10/11/econobel2021' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
print(page.title) # Получаем тег title, отображающийся на вкладке браузера
print(page.title.text) # Выводим текст из полученного тега, который содержится в атрибуте text

<title>Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей</title>
Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей


Eсли при запросе к сайту, а затем при его разборе с помощью BeautifulSoup в тексте страницы не находится нужный тег, попробуйте вывести на печать пару тысяч символов текста страницы. Если там обнаружится нечто похожее на капчу, возможно, сайт посчитал вас роботом и отказывается выдавать содержимое. Чтобы получить его, попробуйте «притвориться» браузером при запросе из скрипта:

requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

# ИЗВЛЕКАЕМ ЗАГОЛОВОК И ВРЕМЯ НАПИСАНИЯ СТАТЬИ

Выполним поставленную ранее задачу: получить информацию о странице и извлечь заголовок статьи, опубликованной на этой странице, дату публикации, а также текст статьи.

Предположим, что мы знаем, что в HTML-коде рассматриваемой нами страницы заголовок статьи заключён в тег h1 … /h1 (заголовок первого уровня).

Тогда мы можем получить его текст с помощью метода find() (с англ. найти) объекта BeautifulSoup, передав ему название интересующего нас тега:


In [33]:
print(page.find('h1').text) # Применяем метод find() к объекту и выводим результат на экран



            Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей
          


Но как же узнать, в каких именно тегах заключена необходимая информация?

Проще всего это сделать с помощью так называемого инструмента разработчика, который есть во всех современных браузерах. Покажем, как открыть данный инструмент на примере использования браузера Google Chrome.

Устанавливаем курсор на элементе страницы (заголовок статьи), информацию о котором хотим получить, нажимаем на правую клавишу мыши и в выпадающем списке выбираем пункт «Просмотреть код элемента» или «Посмотреть код» в зависимости от браузера.

В открывшемся окне инструмента разработчика видим, что информация о заголовке статьи заключена в теге h1 … /h1.
Аналогично получим информации о теге, который содержит дату написания статьи, отображаемую в левом верхнем углу страницы.
Итак, нам нужен тег <span> … </span>.

Теперь получим данные из него с помощью уже известного метода find(), передав название нужного тега:



In [38]:
print(page.find('span').text)


13:04



# НЕУНИКАЛЬНЫЕ ТЕГИ: ИЗВЛЕКАЕМ ТЕКСТ СТАТЬИ

Теперь получим сам текст статьи. Как вы уже знаете, первым делом необходимо определить, в какой тег он заключён. Применим, как и ранее, инструмент разработчика. Метод find() нашёл первый из них, но это не то, что нам надо.

Посмотрим на нашу страницу, используя инструмент разработчика, ещё раз. Можем заметить, что у искомого текста есть свой класс — n1_material text-18 :
Передадим название класса в метод find() с помощью аргумента class_ и получим текст статьи:

In [40]:
print(page.find('div', class_='n1_material text-18').text)
#В данном случае происходит поиск точного строкового значения class атрибута, т. е. выполнение строк кода
#даст одинаковый результат.

print(page.find('div', class_='n1_material').text)
print(page.find('div', class_='n1_material text-18').text)

Премия Шведского национального банка по экономическим наукам памяти Альфреда Нобеля за 2021 год присуждена Дэвиду Карду (David Card) за его вклад в эмпирические исследования экономики рынка труда, а также Джошуа Энгристу (Joshua Angrist) и Гвидо Имбенсу (Guido Imbens) за их вклад в методологию анализа причинно-следственных связей. Прямая трансляция церемонии объявления лауреатов шла на официальном сайте Нобелевской премии.
Премия Шведского национального банка по экономическим наукам памяти Альфреда Нобеля за 2021 год присуждена Дэвиду Карду (David Card) за его вклад в эмпирические исследования экономики рынка труда, а также Джошуа Энгристу (Joshua Angrist) и Гвидо Имбенсу (Guido Imbens) за их вклад в методологию анализа причинно-следственных связей. Прямая трансляция церемонии объявления лауреатов шла на официальном сайте Нобелевской премии.
Премия Шведского национального банка по экономическим наукам памяти Альфреда Нобеля за 2021 год присуждена Дэвиду Карду (David Card) за его вклад 

## СБОР НЕСКОЛЬКИХ ЭЛЕМЕНТОВ: СОБИРАЕМ ВСЕ ССЫЛКИ НА СТРАНИЦЕ

Рассмотрим ещё один сценарий: вы хотите собрать сразу несколько элементов со страницы. Например, представьте, что вы хотите получить названия всех языков программирования, упомянутых на странице в Wikipedia в статье про языки программирования.

Можно заметить, что все названия языков программирования на этой странице связаны ссылками c соответствующими статьями о них. Таким образом, нам необходимо собрать все ссылки на странице. Для ссылок в HTML предусмотрен тег <a> … </a>. Попробуем использовать find():

In [41]:
url = 'https://en.wikipedia.org/wiki/List_of_programming_languages' # Задаём адрес ресурса
response = requests.get(url) # Делаем GET-запрос к ресурсу
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup
print(page.find('a'))

<a id="top"></a>


Мы получили только одну ссылку, хотя на странице их явно больше.

Это происходит, потому что метод 
#### find() 
возвращает только первый подходящий элемент. Если требуется получить больше элементов, необходимо воспользоваться методом find_all() (с англ. найти все):

In [42]:
links = page.find_all('a') # Ищем все ссылки на странице и сохраняем в переменной links в виде списка
print(len(links))
# Не все ссылки соответствуют названиям языков программирования — страница содержит также «служебные» ссылки,
#  такие, например, как Jump to navigation (с англ. Перейти к навигации) или Alphabetical (с англ. По алфавиту):

print([link.text for link in links[0:10]]) # Выводим ссылки с 1 по 9 включительно

943
['', 'Jump to navigation', 'Jump to search', 'Programming languagelists', 'Alphabetical', 'Categorical', 'Chronological', 'Generational', 'v', 't']
